In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import shutil
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/Datasets/archive.zip" -d "/content/dataset"

Archive:  /content/drive/MyDrive/Datasets/archive.zip
replace /content/dataset/MSR-LA - 3467.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls /content/dataset

In [ ]:
!ls /content/dataset/PetImages

In [ ]:
import os
from PIL import Image

dataset_path = "/content/dataset/PetImages"

for folder in ["Cat", "Dog"]:
    folder_path = os.path.join(dataset_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        try:
            img = Image.open(file_path)
            img.verify()
        except:
            os.remove(file_path)

print("Corrupted images removed ✅")

In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(150,150),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(150,150),
    batch_size=32
)

In [ ]:
TRAIN_DIR = "./DATASE"
ORG_DIR = "/content/dataset/PetImages"
CLASS = ['Cat', 'Dog']


In [ ]:
import os
import glob
import shutil

for C in CLASS:
    DEST = os.path.join(TRAIN_DIR, C)

    if not os.path.exists(DEST):
        os.makedirs(DEST)

    for img_path in glob.glob(os.path.join(ORG_DIR, C, "*")):
        SRC = img_path
        shutil.copy(SRC, DEST)

print("Images copied successfully")

Model Building

In [ ]:
!pip install keras

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
base_model = InceptionV3(input_shape=(256, 256, 3), include_top = False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units=2, activation='sigmoid')(X)

model = Model(base_model.input, X)
model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
model.summary()

Pre-Process data using Data Generator

In [ ]:
'''train_datagen = ImageDataGenerator(featurewise_center= True,
                                   rotation_range=0.4,
                                   width_shift_range=0.3,
                                   horizontal_flip= True,
                                   preprocessing_function= preprocess_input,
                                   zoom_range=0.4,
                                   shear_range=0.4)'''

train_data = train_datagen.flow_from_directory(directory ="/content/DATASE",
                                               target_size=(256, 256),
                                               batch_size=64)
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    horizontal_flip=True,
    preprocessing_function=preprocess_input,
    zoom_range=0.4,
    shear_range=0.4
)

In [ ]:
train_data.class_indices

Visualizing the data

In [ ]:
t_img, label = next(train_data)

In [ ]:
def plotImages(img_arr, label):
    for idx, img in enumerate(img_arr):
        if idx <= 10:
            plt.figure(figsize=(4,4))

            # Convert back from Inception preprocessing
            img = (img * 0.5) + 0.5

            plt.imshow(img)
            plt.axis =False
            plt.show()

In [ ]:
plotImages(t_img, label)

In [ ]:
'''from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(
    filepath="./best_model.keras",   # use .keras (recommended now)
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True
)

es = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.01,
    patience=5,
    verbose=1
)

cb = [mc, es]

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    "/content/dataset/PetImages",
    target_size=(224,224),
    batch_size=64,
    class_mode="binary",
    subset="training"
)

val_data = train_datagen.flow_from_directory(
    "/content/dataset/PetImages",
    target_size=(224,224),
    batch_size=64,
    class_mode="binary",
    subset="validation"
)
#model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
#callbacks
mc = ModelCheckpoint(
    "best_model.keras",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=1
)

es = EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    patience=2,
    restore_best_weights=True,
    verbose=1
)
#training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=2
)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("best_model.keras")
model.summary()